In [19]:
import sys; sys.path.append('../../')
import numpy as np
from DataPreparation.DataPreparation import read_data
from utils import load_hyperparameters, save_model, get_metrics
from sklearn import svm
from sklearn.metrics import classification_report
from mlpath import mlquest as mlq
from ModelAnalysis import cross_validation
import pickle
model_name = 'SVM'

mlq.start_quest(model_name, table_dest="../../", log_defs=True)

# <font color="aqua">SVM</font> Model

### Read the data

In [20]:
# x_data_d, y_data_d = mlq.l(read_data)(encode='Frequency', selected=False)
x_data_d, y_data_d = mlq.l(read_data)(kind='Numerical', selected=False )

# x_data_d =x_data_d [['Phys_Act','Height','Weight']]
opt_params = load_hyperparameters(model_name)

### Initiate model

In [21]:
# clf = mlq.l(svm.SVC)(**opt_params)
clf = mlq.l(svm.SVC)(kernel='linear', C=1186.866)

In [22]:
print(opt_params)

{'C': 1186.865625052541, 'gamma': 0.551065306122449, 'kernel': 'linear'}


### Evaluate Model Bias

In [23]:
clf.fit(x_data_d, y_data_d)
y_pred = clf.predict(x_data_d)
train_metrics = classification_report(y_data_d, y_pred, digits=3)
train_acc, train_wf1 = get_metrics(train_metrics)
print(train_metrics)

              precision    recall  f1-score   support

           0      0.987     0.993     0.990       152
           1      0.981     0.975     0.978       160
           2      0.991     0.994     0.992       324
           3      1.000     0.998     0.999       544

    accuracy                          0.993      1180
   macro avg      0.990     0.990     0.990      1180
weighted avg      0.993     0.993     0.993      1180



In [24]:
# perecentage of support vectors
sv =int( np.sum(clf.n_support_)) #/ len(x_data_d)
print("Percentage of support vectors: ",sv)
print("number of support vectors: ", np.sum(clf.n_support_))

Percentage of support vectors:  55
number of support vectors:  55


### Evaluate Model Generalization

In [25]:
kfold=cross_validation(clf, x_data_d, y_data_d, k=[10], n_repeats=[10], random_state=1,loo=False)

val_wf1 = kfold.get(f'10-Repeated 10-fold')[0] 
# repeated_10fold_report= kfold.get(f'10-Repeated 10-fold')[1]
# print(f'10-Repeated 10-fold Report:\n {repeated_10fold_report}')

# loo_wf1=loo.get('loo_wf1')
# loo_report=loo.get('loo_report')
# print(f'Leave-One-Out Report:\n {loo_report}')

10-Repeated 10-fold
0.9865


#### Save Model

In [26]:
save_model(model_name, clf)

### Tracking

In [27]:
mlq.log_metrics(train_acc, train_wf1, val_wf1, sv)
mlq.end_quest()
mlq.show_logs("SVM", table_dest="../../", last_k=10)